
Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [21]:
import pandas as pd
import numpy as np
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

In [123]:
def date_sorter():
    df0 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    df1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    df2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    df3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    df4 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    df5 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    df6 = df.str.extractall(r'^(\d{4})[^0-9]')


    df1 = pd.concat([df0,df1])
    df1 = df1.reset_index()
    df2 = df2.reset_index()
    df3 = df3.reset_index()
    df4 = df4.reset_index()
    df5 = pd.concat([df5,df6])
    df5 = df5.reset_index()


    index_df1 = df1['level_0']
    index_df2 = df2['level_0']
    index_df3 = df3['level_0']
    index_df4 = df4['level_0']
    index_df5 = df5['level_0']


    filter_df2 = []
    for i in index_df2:
        if not((i in index_df1.values)):
            filter_df2.append(i)
    df2 = df2[df2['level_0'].isin(filter_df2)]

    filter_df3 = []
    for i in index_df3:
        if not((i in index_df1.values) |(i in index_df2.values)):
            filter_df3.append(i)
    df3 = df3[df3['level_0'].isin(filter_df3)]

    filter_df4 = []
    for i in index_df4:
        if not((i in index_df1.values) |(i in index_df2.values) | (i in index_df3.values)):
            filter_df4.append(i)
    df4 = df4[df4['level_0'].isin(filter_df4)]

    filter_df5 = []
    for i in index_df5:
        if not((i in index_df1.values) |(i in index_df2.values) | (i in index_df3.values) | (i in index_df4.values)):
            filter_df5.append(i)
    df5 = df5[df5['level_0'].isin(filter_df5)]





    df1.columns=['level_0','match','month','day','year']
    df1['year']=df1['year'].apply(str)
    df1['year']=df1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)

    df2[1] = df2[1].apply(lambda x: x.replace(',',''))
    df2['day'] = df2[1].apply(lambda x:x.split(' ')[0])
    df2['year'] = df2[1].apply(lambda x:x.split(' ')[1])
    df2.columns=['level_0','match','month','day-year','day','year']
    df2 = df2.drop('day-year',axis=1) 


    df3.columns=['level_0','match','day','month','year']
    df3['day'] = df3['day'].replace(np.nan,1000)
    df3['day'] = df3['day'].apply(lambda x: 1 if int(x) == 1000 else x)
    df3['month'] = df3.month.apply(lambda x: x[:3])
    df3['month'] = pd.to_datetime(df3.month, format='%b').dt.month

    df4.columns=['level_0','match','month','year']
    df4['day']=1

    df5.columns=['level_0','match','year']
    df5['day']=1
    df5['month']=1

    final = pd.concat([df1,df2,df3,df4,df5])
    final['date'] =pd.to_datetime(final['month'].apply(str)+'/'+final['day'].apply(str)+'/'+final['year'].apply(str))
    final = final.sort_values(by='level_0').set_index('level_0')
    date_store = final['date']
    return pd.Series([i[0] for i in sorted(enumerate(date_store), key=lambda x:x[1])],np.arange(500))
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64